In [1]:
import pandas as pd
import numpy as np

directory = "../Sesiones/Grabaciones_Sujetos_De_Prueba/05-03-2024/"

eeg_files = ["../Sesiones/Grabaciones_Grupo_TFG/08-04-2024/Rayo_Hollow/EEG/subj-1_ses-S001_task-Rayo_Hollow_run-002_20240408_191356_eeg_ae40ee72-87a6-4cc0-8e81-d89d767ca304-raw.csv"]
emotibit_files = ["../Sesiones/Grabaciones_Grupo_TFG/08-04-2024/Rayo_Hollow/Emotibit/2024-04-08_19-13-52-358527.csv"]

tableapp_emotions_files = ["../Sesiones/Grabaciones_Grupo_TFG/08-04-2024/Rayo_Hollow/TableApp/Rayo_HollowKnight_EmotionsTake200.csv"]
tableapp_events_files = []

subject_id = 3
take = 1
date = "08-04-2024"

### ============================= EEG============================= 

eeg_combined = None  # Inicializar el DataFrame combinado como None
# TimeStamp: 20.0
eeg_start_time = 0

for eeg_file in eeg_files:
    eeg_dataframe = pd.read_csv(eeg_file)  # Leer el archivo CSV y cargarlo como un DataFrame

    # ============================= Procesar EEG =============================
    # Dropear la primera columna
    eeg_dataframe = eeg_dataframe.iloc[:, 1:]

    # Añadir columnas INFO experimento
    eeg_dataframe.insert(loc=1, column="Subject ID", value=subject_id)
    eeg_dataframe.insert(loc=2, column="Take", value=take)
    eeg_dataframe.insert(loc=3, column="Date", value=date)

    # ============================= Concatenar dataframes =============================
    if eeg_combined is None:
        eeg_combined = eeg_dataframe  # Asignar el primer DataFrame directamente
    else:
        eeg_combined = pd.concat([eeg_combined, eeg_dataframe], ignore_index=True)  # Concatenar los DataFrames sucesivos


    # Convertir la columna de tiempo a segundos (si está en formato de tiempo)
    if eeg_combined['time'].dtype == 'object':
        eeg_combined['time'] = pd.to_datetime(eeg_combined['time']).astype(float) / 10**9

    # Filtrar las filas con tiempo mayor o igual al offset
    eeg_combined = eeg_combined[eeg_combined['time'] >= eeg_start_time]

    # Restar el tiempo offset a las filas restantes
    eeg_combined['time'] -= eeg_start_time

    # Resetear el índice del DataFrame
    eeg_combined = eeg_combined.reset_index(drop=True)


display(eeg_combined)

FileNotFoundError: [Errno 2] No such file or directory: '../Sesiones/Grabaciones_Grupo_TFG/08-04-2024/Rayo_Hollow/EEG/subj-1_ses-S001_task-Rayo_Hollow_run-002_20240408_191356_eeg_ae40ee72-87a6-4cc0-8e81-d89d767ca304-raw.csv'

In [ ]:


### ============================= EMOTIBIT =============================
import csv


emotibit_combined = None  # Inicializar el DataFrame combinado como None

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", None)

# Definir el numero de columnas
emotibit_columns = ['time', 'AX', 'AY', 'AZ', 'GX', 'GY', 'GZ', 'T1', 'MX', 'MY', 'MZ', 'HR', 'BI', 'PI', 'PR', 'PG', 'EA', 'EL', 'ER', 'SA', 'SR', 'SF']

for emotibit_file in emotibit_files:
    
    datos = []
    with open(emotibit_file, 'r', newline='') as archivo:
        lector_csv = csv.reader(archivo)
        for fila in lector_csv:
            try:
                datos.append(fila)
            except Exception as e:
                print(f"Error en la fila {fila}: {e}")
                
    # Convertir la lista de datos en un DataFrame
    emotibit_dataframe = pd.DataFrame(datos)
    # ============================= Procesar EMOTIBIT =============================
    # TimeStamp: 179844



    # Añadir columnas INFO experimento
    emotibit_dataframe.insert(loc=0, column="Subject ID", value=subject_id)
    emotibit_dataframe.insert(loc=1, column="Take", value=take)
    emotibit_dataframe.insert(loc=2, column="Date", value=date)

    # Renombrar el header de la sexta columna
    nuevo_nombre = 'time'
    emotibit_dataframe = emotibit_dataframe.rename(columns={0: nuevo_nombre})


    # Drop unneccesary data
    emotibit_dataframe = emotibit_dataframe.drop(emotibit_dataframe[emotibit_dataframe[3] == 'AK'].index)
    emotibit_dataframe = emotibit_dataframe.drop(emotibit_dataframe[emotibit_dataframe[3] == 'RB'].index)
    emotibit_dataframe = emotibit_dataframe.drop(emotibit_dataframe[emotibit_dataframe[3] == 'TL'].index)
    emotibit_dataframe = emotibit_dataframe.drop(emotibit_dataframe[emotibit_dataframe[3] == 'RD'].index)
    emotibit_dataframe = emotibit_dataframe.drop(emotibit_dataframe[emotibit_dataframe[3] == 'TH'].index)
    emotibit_dataframe = emotibit_dataframe.drop(emotibit_dataframe[emotibit_dataframe[3] == 'EM'].index)
    emotibit_dataframe = emotibit_dataframe.drop(emotibit_dataframe[emotibit_dataframe[3] == 'DC'].index)
    emotibit_dataframe = emotibit_dataframe.drop(emotibit_dataframe[emotibit_dataframe[3] == 'DO'].index)

    # Obtain Emotibit start time
    EmotibitStartTime = 26519

    # Convertir la columna de tiempo a segundos (si está en formato de tiempo)
    if emotibit_dataframe['time'].dtype == 'object':
        emotibit_dataframe['time'] = emotibit_dataframe['time'].astype(float)

    if emotibit_dataframe[2].dtype == 'object':
        emotibit_dataframe[2] = emotibit_dataframe[2].astype(int)

        
    # Convert Emotibit time values (miliseconds) to seconds and add the offset of the start time
    # Result is EmotibitTime -> s (EEG)
    emotibit_dataframe['time'] = emotibit_dataframe['time'].apply(lambda x:  x - EmotibitStartTime)
    emotibit_dataframe['time'] = emotibit_dataframe['time'].apply(lambda x:  x/1000)

    # Filtrar las filas con tiempo mayor o igual al offset
    emotibit_dataframe = emotibit_dataframe[emotibit_dataframe['time'] >= 0]

    # Agrupar por el valor de tiempo (columna 3)
    grupos_tiempo = emotibit_dataframe.groupby('time')
    final_emotibit_dataframe_data = pd.DataFrame(columns=emotibit_columns)

    ### ITERADOR DE GRUPOS
    for tiempo, grupo in grupos_tiempo:
        tempDataFrame = pd.DataFrame(columns=emotibit_columns)

        contador = 0
        ### ITERADOR DE FILAS
        for index, row in grupo.iterrows():
            
            # Obtener el valor de la fila en la columna actual
            groupTag = grupo[3][index]

            numberOfPackets = grupo[2][index]

            groupValue = grupo[6][index]

           
            # Calcular la media para la primera fila y columnas especificadas
            groupValueDf = grupo.iloc[contador, 9:(9 + numberOfPackets)]
            groupValueDf = groupValueDf.astype(float)
            groupValue = groupValueDf.mean() 

            contador += 1

            # Asignar time
            tempDataFrame.at[0, 'time'] =  grupo['time'][index]

            ### ITERADOR PARA SABER EL TAG
            for column in tempDataFrame.columns:
                if groupTag == column:
                    tempDataFrame.at[0, column] = groupValue

        # pasar tempDataFrame a finalDataFrame
        final_emotibit_dataframe_data = pd.concat([final_emotibit_dataframe_data, tempDataFrame], ignore_index=True)   
    
    


    final_emotibit_dataframe_data = final_emotibit_dataframe_data.sort_values(by='time')
    
    # Añadir columnas INFO experimento
    final_emotibit_dataframe_data.insert(loc=1, column="Subject ID", value=subject_id)
    final_emotibit_dataframe_data.insert(loc=2, column="Take", value=take)
    final_emotibit_dataframe_data.insert(loc=3, column="Date", value=date)

    # ============================= Concatenar dataframes =============================
    if emotibit_combined is None:
        emotibit_combined = final_emotibit_dataframe_data  # Asignar el primer DataFrame directamente
    else:
        emotibit_combined = pd.concat([emotibit_combined, final_emotibit_dataframe_data], ignore_index=True)  # Concatenar los DataFrames sucesivos

display(emotibit_combined)

In [ ]:


### ============================= TABLEAPP =============================

tableapp_emotions_combined = None

file_cont = 0

for tableapp_emotions_file in tableapp_emotions_files:
    tableapp_emotions_dataframe = pd.read_csv(tableapp_emotions_file)  # Leer el archivo CSV y cargarlo como un DataFrame

    # ============================= Procesar TABLEAPP =============================
    # Redondear a 3 decimales
    # tableapp_emotions_dataframe['time'] = tableapp_emotions_dataframe['time'].round(3)

    # Renombrar headers
    nuevo_nombre = 'Focus' + str(file_cont + 1)
    tableapp_emotions_dataframe = tableapp_emotions_dataframe.rename(columns={tableapp_emotions_dataframe.columns[1]: nuevo_nombre})

    nuevo_nombre = 'Amusement' + str(file_cont + 1)
    tableapp_emotions_dataframe = tableapp_emotions_dataframe.rename(columns={tableapp_emotions_dataframe.columns[2]: nuevo_nombre})

    nuevo_nombre = 'Surprise' + str(file_cont + 1)
    tableapp_emotions_dataframe = tableapp_emotions_dataframe.rename(columns={tableapp_emotions_dataframe.columns[3]: nuevo_nombre})

    nuevo_nombre = 'Fright' + str(file_cont + 1)
    tableapp_emotions_dataframe = tableapp_emotions_dataframe.rename(columns={tableapp_emotions_dataframe.columns[4]: nuevo_nombre})

    nuevo_nombre = 'Inmersion' + str(file_cont + 1)
    tableapp_emotions_dataframe = tableapp_emotions_dataframe.rename(columns={tableapp_emotions_dataframe.columns[5]: nuevo_nombre})

    # ============================= Concatenar dataframes =============================
    if tableapp_emotions_combined is None:
        tableapp_emotions_combined = tableapp_emotions_dataframe  # Asignar el primer DataFrame directamente
    else:
        tableapp_emotions_combined = pd.concat([tableapp_emotions_combined.set_index('time'), tableapp_emotions_dataframe.set_index('time')], axis=1).reset_index()

    tableapp_emotions_combined = tableapp_emotions_combined.sort_values(by='time')

    file_cont += 1

    # Eliminar la segunda columna "subject ID"
    # tableapp_emotions_combined = tableapp_emotions_combined.loc[:, ~tableapp_emotions_combined.columns.duplicated()]


# Dar las vueltas necesarias
while file_cont < 3:
    # Crear el diccionario de datos con valores NaN
    data = {
        'time': [],
        'Focus': [],
        'Amusement': [],
        'Surprise': [],
        'Fright': [],
        'Inmersion': []
    }

    # Crear el DataFrame a partir del diccionario
    tableapp_emotions_dataframe = pd.DataFrame(data)

    # ============================= Procesar TABLEAPP =============================
    # Redondear a 3 decimales
    # tableapp_emotions_dataframe['time'] = tableapp_emotions_dataframe['time'].round(3)

    # Renombrar headers
    nuevo_nombre = 'Focus' + str(file_cont + 1)
    tableapp_emotions_dataframe = tableapp_emotions_dataframe.rename(columns={tableapp_emotions_dataframe.columns[1]: nuevo_nombre})

    nuevo_nombre = 'Amusement' + str(file_cont + 1)
    tableapp_emotions_dataframe = tableapp_emotions_dataframe.rename(columns={tableapp_emotions_dataframe.columns[2]: nuevo_nombre})

    nuevo_nombre = 'Surprise' + str(file_cont + 1)
    tableapp_emotions_dataframe = tableapp_emotions_dataframe.rename(columns={tableapp_emotions_dataframe.columns[3]: nuevo_nombre})

    nuevo_nombre = 'Fright' + str(file_cont + 1)
    tableapp_emotions_dataframe = tableapp_emotions_dataframe.rename(columns={tableapp_emotions_dataframe.columns[4]: nuevo_nombre})

    nuevo_nombre = 'Inmersion' + str(file_cont + 1)
    tableapp_emotions_dataframe = tableapp_emotions_dataframe.rename(columns={tableapp_emotions_dataframe.columns[5]: nuevo_nombre})

    # ============================= Concatenar dataframes =============================
    if tableapp_emotions_combined is None:
        tableapp_emotions_combined = tableapp_emotions_dataframe  # Asignar el primer DataFrame directamente
    else:
        tableapp_emotions_combined = pd.concat([tableapp_emotions_combined.set_index('time'), tableapp_emotions_dataframe.set_index('time')], axis=1).reset_index()

    tableapp_emotions_combined = tableapp_emotions_combined.sort_values(by='time')

    file_cont += 1


# Añadir INFO Experimento
tableapp_emotions_combined.insert(loc=1, column="Subject ID", value=subject_id)
tableapp_emotions_combined.insert(loc=2, column="Take", value=take)
tableapp_emotions_combined.insert(loc=3, column="Date", value=date)

tableapp_events_combined = None
file_cont = 0

for tableapp_events_file in tableapp_events_files:
    tableapp_events_dataframe = pd.read_csv(tableapp_events_file)  # Leer el archivo CSV y cargarlo como un DataFrame

    # ============================= Procesar TABLEAPP =============================
    # Redondear a 3 decimales
    # tableapp_events_dataframe['time'] = tableapp_events_dataframe['time'].round(3)

    # Dropear la ultima columna
    tableapp_events_dataframe = tableapp_events_dataframe.iloc[:, :-1]

    # Renombrar headers
    nuevo_nombre = 'Evento' + str(file_cont + 1)
    tableapp_events_dataframe = tableapp_events_dataframe.rename(columns={tableapp_events_dataframe.columns[1]: nuevo_nombre})
        
    tableapp_events_dataframe.insert(loc=1, column="Subject ID", value=subject_id)
    tableapp_events_dataframe.insert(loc=2, column="Take", value=take)
    tableapp_events_dataframe.insert(loc=3, column="Date", value=date)

    # ============================= Concatenar dataframes =============================
    if tableapp_events_combined is None:
        tableapp_events_combined = tableapp_events_dataframe  # Asignar el primer DataFrame directamente
    else:
        tableapp_events_combined = pd.concat([tableapp_events_combined, tableapp_events_dataframe], ignore_index=True)  # Concatenar los DataFrames sucesivos

    file_cont += 1


# Dar las vueltas necesarias
while file_cont < 3:
    # Crear el diccionario de datos con valores NaN
    data = {
        'time': [],
        'Evento': []
    }

    # Crear el DataFrame a partir del diccionario
    tableapp_events_dataframe = pd.DataFrame(data)

    # ============================= Procesar TABLEAPP =============================
    # Redondear a 3 decimales
    # tableapp_events_dataframe['time'] = tableapp_events_dataframe['time'].round(3)

    # Renombrar headers
    nuevo_nombre = 'Evento' + str(file_cont + 1)
    tableapp_events_dataframe = tableapp_events_dataframe.rename(columns={tableapp_events_dataframe.columns[1]: nuevo_nombre})
        
    tableapp_events_dataframe.insert(loc=1, column="Subject ID", value=subject_id)
    tableapp_events_dataframe.insert(loc=2, column="Take", value=take)
    tableapp_events_dataframe.insert(loc=3, column="Date", value=date)

    # ============================= Concatenar dataframes =============================
    if tableapp_events_combined is None:
        tableapp_events_combined = tableapp_events_dataframe  # Asignar el primer DataFrame directamente
    else:
        tableapp_events_combined = pd.concat([tableapp_events_combined, tableapp_events_dataframe], ignore_index=True)  # Concatenar los DataFrames sucesivos

    file_cont += 1

# display(tableapp_emotions_combined)
# display(tableapp_events_combined)

# display(final_dataframe)

In [ ]:
### ============================= Concatenar =============================
eeg_emotibit_final_dataframe = pd.concat([eeg_combined, emotibit_combined], ignore_index=True)  # Concatenar los DataFrames sucesivos
tableapp_final_dataframe = pd.concat([tableapp_emotions_combined, tableapp_events_combined], ignore_index=True)  # Concatenar los DataFrames sucesivos

tableapp_final_dataframe = pd.concat([eeg_emotibit_final_dataframe, tableapp_final_dataframe], ignore_index=True)  # Concatenar los DataFrames sucesivos

### ============================= Dropear final tabla por tiempo =============================
# Filtrar las filas con tiempo menor o igual al end_time
end_time = 972.092
tableapp_final_dataframe = tableapp_final_dataframe[tableapp_final_dataframe['time'] <= end_time]

# Sort table by ascendant order (depending on time)
tableapp_final_dataframe = tableapp_final_dataframe.sort_values(by='time')

display(tableapp_final_dataframe)

In [ ]:
### ============================= Exportar CSV =============================
# Agrupar por las columnas 'time', 'Subject ID', 'Take', 'Date' y aplicar la función lambda para cada columna
tableapp_final_dataframe = tableapp_final_dataframe.groupby(['time', 'Subject ID', 'Take', 'Date']).agg(lambda x: next((i for i in x if not pd.isnull(i)), np.nan)).reset_index()


# Gender - masculine 0
# Gender - femenine 1
# Gender - non binary 2

# Experience with videogames - Mucha, juego videojuegos desde hace años y es mi entretenimiento principal - 0
# Experience with videogames - Bastante, es un hobbie para mí pero no es mi entretenimiento principal - 1
# Experience with videogames - Media, juego de vez en cuando para desconectar - 2
# Experience with videogames - Poca, sé lo básico pero no me interesa mucho el tema - 3
# Experience with videogames - Ninguna, no suelo jugar ni ver nada relacionado con los videojuegos - 4

# Play time - Entre 0 y 5 horas - 0
# Play time - Entre 5 y 10 horas - 1
# Play time - Entre 10 y 15 horas - 2
# Play time - Entre 15 y 20 horas - 3
# Play time - Más de 20 horas - 4

# Familiarized with horror games - Sí, me gusta jugarlos - 0
# Familiarized with horror games - Sí, me gusta ver jugar a otros - 1
# Familiarized with horror games - No - 2

# Insertar demografia
tableapp_final_dataframe.insert(loc=4, column="Age", value=21)
tableapp_final_dataframe.insert(loc=5, column="Gender", value=0)
tableapp_final_dataframe.insert(loc=6, column="ExperienceWithGames", value=0)
tableapp_final_dataframe.insert(loc=7, column="PlayTime", value=1)
tableapp_final_dataframe.insert(loc=8, column="FamiliarizedWithHorrorGames", value=0)

# # Combina la ruta de la carpeta y el nombre del archivo para obtener la ruta completa
file_path = "Rayo_Hollow_Take1_Raw.csv"

# # Exporta el DataFrame a un archivo CSV en la ruta especificada
tableapp_final_dataframe.to_csv(file_path, index=False)  # Si no deseas guardar el índice del DataFrame, puedes establecer index=False